### Библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from tqdm import tqdm

import statsmodels.api as sm
from scipy.stats import t, f, boxcox, skew, kurtosis, gmean
from statsmodels.stats.diagnostic import linear_reset, het_white

import warnings
warnings.filterwarnings('ignore')

### Обработка

In [2]:
data = pd.read_csv('data_after_processing.csv', encoding='utf-8')

data = data.drop(['title', 'Unnamed: 0'], axis=1)
data = data.drop(['author_Другой', 'publisher_Другой', 'publication_year_Другой',
                  'cover_type_Мягкий заламинированный картон', 'reading_age_6+'], axis=1)

### Логарифм цены

In [3]:
data['log_price'] = np.log(data['price'])
data = data.drop(['price'], axis=1)
y = data['log_price']
X = data.drop(['log_price'], axis=1)

### Регрессоры, которые можно логарифмировать и нет

In [4]:
cols_to_try_log = [
    'avg_rating', 'cnt_reviews', 'pages_cnt', 'tirage', 
    'weight', 'thickness', 'width', 'length', 'volume'
]

unconditional_cols = [x for x in X.columns.tolist() if x not in cols_to_try_log]

In [5]:
def generate_all_combinations(num_of_repeats):
    """Генерирует все комбинации признаков, к которым применяется функциональное преобразование"""
    return product([False, True], repeat=num_of_repeats)

def prepare_X(cols_to_try_func, combination, df, func):
    """Создает матрицу X для текущей комбинации"""
    """Примеры функций: np.log, np.power, np.reciprocal, ..."""
    X_temp = df[unconditional_cols].copy()
    feature_name = getattr(func, '__name__', repr(func))
    
    for i, col in enumerate(cols_to_try_func):
        if combination[i]:
            try:
                safe_col = df[col]
                if (safe_col <= 0).any() and (func == np.log) or (func == np.reciprocal):
                    safe_col = safe_col + 1e-6
                X_temp[f'{feature_name}_{col}'] = func(safe_col)
            except Exception as e:
                print(f"Ошибка при применении функции к {col}: {e}")
        else:
            X_temp[col] = df[col]
    
    return sm.add_constant(X_temp)

def getting_statistic(cols_to_try_func, df, func):
    results = []

    total_combinations = 2 ** len(cols_to_try_func)
    feature_name = getattr(func, '__name__', repr(func))

    for combination in tqdm(generate_all_combinations(num_of_repeats=len(cols_to_try_func)), total=total_combinations):
        try:
            # Подготавливаем данные
            X_curr = prepare_X(cols_to_try_func, combination, df, func)

            model = sm.OLS(df['log_price'], X_curr).fit()
            results.append({
                'combination': combination,
                'aic': model.aic,
                'bic': model.bic,
                'adj_r2': model.rsquared_adj,
            })

        except Exception as e:
            print(f"Error in combination {combination}: {str(e)}")
            continue

    df_results = pd.DataFrame(results)

    df_results[f'{feature_name}_columns'] = df_results['combination'].apply(
        lambda x: [cols_to_try_func[i] for i, my_func in enumerate(x) if my_func]
    )
    
    return df_results

In [6]:
# Проверяем логарифмирование
data_frame = getting_statistic(cols_to_try_func=cols_to_try_log, df=data, func=np.log)

best_aic = data_frame.loc[data_frame['aic'].idxmin()]
best_bic = data_frame.loc[data_frame['bic'].idxmin()]
best_adj_r2 = data_frame.loc[data_frame['adj_r2'].idxmax()]

print("Лучшая модель по AIC:")
print(f"Логарифмированные переменные: {best_aic['log_columns']}")
print(f"AIC: {best_aic['aic']:.2f}\n")

print("Лучшая модель по BIC:")
print(f"Логарифмированные переменные: {best_bic['log_columns']}")
print(f"BIC: {best_bic['bic']:.2f}\n")

print("Лучшая модель по Adj.R²:")
print(f"Логарифмированные переменные: {best_adj_r2['log_columns']}")
print(f"Adj.R²: {best_adj_r2['adj_r2']:.4f}")

100%|█████████████████████████████████████████| 512/512 [00:07<00:00, 72.06it/s]


Лучшая модель по AIC:
Логарифмированные переменные: ['tirage']
AIC: -297.99

Лучшая модель по BIC:
Логарифмированные переменные: ['tirage']
BIC: -84.86

Лучшая модель по Adj.R²:
Логарифмированные переменные: ['tirage']
Adj.R²: 0.8021


In [7]:
# Проверяем признаки вида 1/x
my_func = np.reciprocal
data_frame = getting_statistic(cols_to_try_func=cols_to_try_log, df=data, func=my_func)

best_aic = data_frame.loc[data_frame['aic'].idxmin()]
best_bic = data_frame.loc[data_frame['bic'].idxmin()]
best_adj_r2 = data_frame.loc[data_frame['adj_r2'].idxmax()]

column_name = f'{my_func.__name__}_columns'

print("Лучшая модель по AIC:")
print(f"Преобразованные переменные: {best_aic[column_name]}")

print(f"AIC: {best_aic['aic']:.2f}\n")

print("Лучшая модель по BIC:")
print(f"Преобразованные переменные: {best_bic[column_name]}")
print(f"BIC: {best_bic['bic']:.2f}\n")

print("Лучшая модель по Adj.R²:")
print(f"Преобразованные переменные: {best_adj_r2[column_name]}")
print(f"Adj.R²: {best_adj_r2['adj_r2']:.4f}")

100%|█████████████████████████████████████████| 512/512 [00:07<00:00, 68.25it/s]

Лучшая модель по AIC:
Преобразованные переменные: []
AIC: -215.04

Лучшая модель по BIC:
Преобразованные переменные: []
BIC: -1.90

Лучшая модель по Adj.R²:
Преобразованные переменные: []
Adj.R²: 0.7970


Также функции будут работать, если мы решим проверить какие-то кастомные функции типо этого:

In [8]:
def quadratic_shift(x):
    return x**2 + 3*x + 5

### Итоговая модель с ln Y

После проверки всех функциональных форм обучаем итоговую модель:

In [9]:
cols_to_log = ['tirage']
cols_not_to_log = [col for col in cols_to_try_log if col not in cols_to_log] + unconditional_cols

X_log = np.log(data.loc[:, cols_to_log])
X_not_to_log = data.loc[:, cols_not_to_log]

X_log_model = sm.add_constant(pd.concat((X_log, X_not_to_log), axis=1))

log_model = sm.OLS(data['log_price'], X_log_model).fit()
log_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     389.4
Date:                Mon, 05 May 2025   Prob (F-statistic):               0.00
Time:                        17:37:26   Log-Likelihood:                 184.00
No. Observations:                3260   AIC:                            -298.0
Df Residuals:                    3225   BIC:                            -84.86
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               5.8943      0.280     21.032      0.000       5.345       6.444
tirage                             -0.1782      0.010    -17.857      0.000      -0.198      -0.159
avg_rating                          0.0731      0.012      6.256      0.000       0.050       0.096
cnt_reviews                         0.0004   3.51e-05     10.025      0.000       0.000       0.000
pages_cnt                          -0.0003   4.17e-05     -7.621      0.000      -0.000      -0.000
weight                              0.0022   4.78e-05     45.115      0.000       0.002       0.002
thickness                           0.1769      0.022      8.167      0.000       0.134       0.219
width                               0.0760      0.005     14.086      0.000       0.065       0.087
length                              0.0247      0.004      6.256      0.000       0.017       0.032
volume                             -0.0012      8e-05    -14.696      0.000      -0.001      -0.001
author_Джейн Остен                  0.0134      0.028      0.469      0.639      -0.042       0.069
author_Джек Лондон                 -0.0741      0.028     -2.678      0.007      -0.128      -0.020
author_Джордж Оруэлл                0.0155      0.029      0.526      0.599      -0.042       0.073
author_Лев Толстой                 -0.1152      0.031     -3.681      0.000      -0.177      -0.054
author_Луиза Мэй Олкотт            -0.0343      0.033     -1.048      0.295      -0.098       0.030
author_Михаил Булгаков              0.0616      0.027      2.285      0.022       0.009       0.114
author_Николай Гоголь              -0.0944      0.033     -2.843      0.004      -0.159      -0.029
author_Федор Достоевский           -0.1019      0.021     -4.813      0.000      -0.143      -0.060
author_Эрих Ремарк                  0.1047      0.030      3.547      0.000       0.047       0.163
publisher_АСТ                      -0.1477      0.030     -4.911      0.000      -0.207      -0.089
publisher_Азбука                   -0.2000      0.031     -6.402      0.000      -0.261      -0.139
publisher_Иностранка               -0.1291      0.036     -3.621      0.000      -0.199      -0.059
publisher_Манн, Иванов и Фербер     0.0127      0.041      0.313      0.754      -0.067       0.092
publisher_Эксмо                    -0.1713      0.030     -5.617      0.000      -0.231      -0.111
publication_year_2021              -0.0061      0.045     -0.133      0.894      -0.095       0.083
publication_year_2022               0.0386      0.038      1.016      0.310      -0.036       0.113
publication_year_2023               0.0400      0.037      1.095      0.274      -0.032       0.112
publication_year_2024               0.0580      0.036      1.614      0.107      -0

**мини-вопрос на подумать:** допустим, мы попробовали разные функциональные формы для призаков и в качестве того, для чего резонно применять `log` и `custom_func` у нас функция выдала один и тот же признак `tirage`. Что с ним делать?
<br>
<br>
**"ответ":** я предлагаю сильно не заморачиваться и выбрать что придется